In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

# Data Loading

In [157]:
data = pd.read_csv("../../data/processed/data.csv",
                   dtype = {
                            "expensa_id": "object",
                            "unidad_id": "object",
                            "propietario_id": "object",
                            "inquilino_id": "object",
                            "consorcio_id": "object",
                            "expensa_padre_id": "object",
                            "unidad_prop": "object",
                            "consorcio_id": "object",
                            "usuario_creador_id": "object",
                            "concepto_id": "object",
                            "mes": "object",
                            "anio": "object",
                            "unidad_denominacion":"object",
                            "consorcio_nombre": "object",
                            "version":"object",
                            "concepto_id": "object",
                            "unidad_tipo": "object",
                            "modo_pago": "object",
                            "forma_prorrateo": "object",
                            "tipo_prorrateo": "object",
                            "metodo_pago": "object",                       
                    },
                    parse_dates = ["fecha_vencimiento_1", "fecha_vencimiento_2"])
data.fecha_pago = pd.to_datetime(data.fecha_pago, errors="coerce")

In [158]:
data.shape

(264541, 27)

In [159]:
data.dtypes

expensa_id                     object
unidad_id                      object
propietario_id                 object
inquilino_id                   object
fecha_vencimiento_1    datetime64[ns]
fecha_vencimiento_2    datetime64[ns]
monto                         float64
monto_total                   float64
monto_parcial                 float64
expensa_padre_id               object
int_dia                       float64
int_mes                       float64
mes                            object
anio                           object
unidad_denominacion            object
unidad_prop                    object
consorcio_id                   object
consorcio_nombre               object
version                        object
concepto_id                    object
unidad_tipo                    object
modo_pago                      object
forma_prorrateo                object
tipo_prorrateo                 object
fecha_pago             datetime64[ns]
monto_pago                    float64
metodo_pago 

# Generate Posible Target Variables

## Impago

In [160]:
data["target"] = (data.fecha_pago.dt.month == data.fecha_vencimiento_1.dt.month) & (data.fecha_pago.dt.year == data.fecha_vencimiento_1.dt.year)  

## Paga En el Mes?

In [161]:
data["fecha_expensa"] = data.anio + '-' + data.mes + '-01'
data.fecha_expensa = pd.to_datetime(data.fecha_expensa)
data["fecha_expensa_anterior"] = data.fecha_expensa - pd.DateOffset(months=1)

In [198]:
data

,expensa_id,unidad_id,propietario_id,inquilino_id,fecha_vencimiento_1,fecha_vencimiento_2,monto,monto_total,monto_parcial,expensa_padre_id,...,unidad_tipo,modo_pago,forma_prorrateo,tipo_prorrateo,fecha_pago,monto_pago,metodo_pago,target,fecha_expensa,fecha_expensa_anterior
0,2588,135,181,0,2014-06-10,2014-06-25,2940.63,122499.07,0.0,0,...,4,mes_vencido,normal,detalle,2014-10-23,4116.88,Efectivo,False,2014-05-01,2014-04-01
1,3372,135,181,0,2014-08-10,2014-08-25,3948.45,122499.07,0.0,0,...,4,mes_vencido,normal,detalle,2014-10-23,4738.14,Efectivo,False,2014-07-01,2014-06-01
2,7643,135,181,0,2014-11-10,2014-11-25,1669.52,80511.69,0.0,0,...,4,mes_vencido,normal,detalle,2014-11-07,1729.62,Rapipago,True,2014-10-01,2014-09-01
3,8880,135,181,0,2014-12-10,2014-12-25,3123.80,148806.68,0.0,0,...,4,mes_vencido,normal,detalle,2014-12-10,3236.26,Rapipago,True,2014-11-01,2014-10-01
4,12923,135,181,0,2015-01-10,2015-01-25,2723.84,127312.34,0.0,0,...,4,mes_vencido,normal,detalle,2015-01-09,2821.90,Rapipago,True,2014-12-01,2014-11-01
5,14658,135,181,0,2015-02-10,2015-02-25,2379.66,112226.71,0.0,0,...,4,mes_vencido,normal,detalle,2015-02-09,2465.33,Rapipago,True,2015-01-01,2014-12-01
6,16270,135,181,0,2015-03-10,2015-03-25,1996.88,94292.58,0.0,0,...,4,mes_vencido,normal,detalle,2015-03-11,2172.21,Rapipago,True,2015-02-01,2015-01-01
7,18947,135,181,0,2015-04-10,2015-04-25,2295.75,108096.11,0.0,0,...,4,mes_vencido,normal,detalle,2015-05-22,2747.05,Rapipago,False,2015-03-01,2015-02-01
8,20472,135,181,0,2015-05-10,2015-05-25,2060.99,97701.72,0.0,0,...,4,mes_vencido,normal,detalle,2015-05-22,2241.95,Rapipago,True,2015-04-01,2015-03-01
9,22581,135,181,0,2015-06-10,2015-06-25,2639.60,123724.35,0.0,0,...,4,mes_vencido,normal,detalle,2015-06-09,2734.63,Rapipago,True,2015-05-01,2015-04-01


In [216]:
def get_historiales(fecha_inicio, fecha_fin, data):
    data_local = data.loc[(data.fecha_expensa >= fecha_inicio) & (data.fecha_expensa <fecha_fin),:]

    #unidades
    pagos_por_unidad = data.groupby("unidad_id").count().reset_index()[["unidad_id","fecha_expensa"]]
    unidades = pagos_por_unidad.loc[pagos_por_unidad.fecha_expensa >= 6, "unidad_id"]
    
    #fechas
    fechas = data_local.fecha_expensa.unique()
    fechas.sort()
    fechas = [fecha[:10] for fecha in list(np.datetime_as_string(fechas))]
    
    #targets
    data_indexed = data_local[["unidad_id", "fecha_expensa", "target"]]
    data_indexed.loc[:,"fecha_expensa"] = data_indexed.fecha_expensa.map(str)
    data_indexed.fecha_expensa = data_indexed.fecha_expensa.map(lambda x: x[:10])
    targets = data_indexed.set_index(["unidad_id","fecha_expensa"]).to_dict()["target"]
    
    #historiales
    historiales = {}
    for unidad in unidades:
        historial = list()
        for fecha in fechas:
            try:
                value = targets[(unidad,fecha)]
            except:
                value = np.nan
            historial.append(value)
        historiales[unidad] = historial
    
    print("cantidad de unidades antes:", len(data.unidad_id.unique()))
    print("cantidad de meses antes:", len(data.fecha_expensa.unique()))
    
    print("cantidad de unidades:", len(unidades))
    print("cantidad meses ahora:", len(fechas))
    
    df = pd.DataFrame.from_dict(historiales, orient="index")
    df.columns = fechas
    
    return df

In [217]:
historiales =  get_historiales("2014-06-01", "2019-01-01", data)

C:\Anaconda3\envs\deeplearning-labs\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Anaconda3\envs\deeplearning-labs\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


cantidad de unidades antes: 16139
cantidad de meses antes: 63
cantidad de unidades: 11418
cantidad meses ahora: 55


In [223]:
historiales.to_csv('../../data/processed/historiales.csv')